# Dependencies

In [1]:
# Import Dependencies
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Matplotlib visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Splitting data into training and testing
from sklearn.model_selection import train_test_split

# Preprocessing library to encode data
from sklearn import preprocessing

# Machine Learning Model
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [2]:
# Import Train and Test Data
train_data = pd.read_csv("train_final_clean_df.csv")
test_data = pd.read_csv("test_final_clean_df.csv")

In [3]:
train_data.head(2)

,source_system_tab,source_screen_name,source_type,song_length,genre_ids,artist_name,composer,language,city,age,gender,registered_via,target,age_group
0,explore,Explore,online-playlist,206471.0,359,Bastille,Dan Smith| Mark Crew,52.0,1,0,Unknown,7,1,0
1,my library,Local playlist more,local-playlist,284584.0,1259,Various Artists,Unknown,52.0,13,24,female,9,1,1


In [4]:
test_data.head(2)

,id,source_system_tab,source_screen_name,source_type,song_length,genre_ids,artist_name,composer,language,city,age,gender,registered_via,age_group
0,0,my library,Local playlist more,local-library,224130.0,458,梁文音 (Rachel Liang),Qi Zheng Zhang,3.0,1,0,Unknown,7,0
1,1,my library,Local playlist more,local-library,320470.0,465,林俊傑 (JJ Lin),林俊傑,3.0,1,0,Unknown,7,0


# Encode label columns

In [5]:
train_encoded_data = train_data.copy()
test_encoded_data = test_data.copy()

# Replace Unseen items in test data
test_encoded_data['source_screen_name'] = test_encoded_data['source_screen_name'].replace('People local','Unknown')
test_encoded_data['source_screen_name'] = test_encoded_data['source_screen_name'].replace('People global','Unknown')
test_encoded_data['genre_ids'] = test_encoded_data['genre_ids'].replace(1089,0)
test_encoded_data['genre_ids'] = test_encoded_data['genre_ids'].replace(166,0)
test_encoded_data['genre_ids'] = test_encoded_data['genre_ids'].replace(765,0)
test_encoded_data['genre_ids'] = test_encoded_data['genre_ids'].replace(1061,0)
test_encoded_data['genre_ids'] = test_encoded_data['genre_ids'].replace(303,0)
test_encoded_data['genre_ids'] = test_encoded_data['genre_ids'].replace(2045,0)


# Select the categorical columns
categorical_columns = ['source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'gender']

i = 0
le = []

for column in categorical_columns:
    le.append(preprocessing.LabelEncoder())
    fit = le[i].fit(train_encoded_data[column])
    transform = le[i].transform(train_encoded_data[column])
    train_encoded_data[column] = transform
    i += 1
    
i = 0

for column in categorical_columns:
    transform = le[i].transform(test_encoded_data[column])
    test_encoded_data[column] = transform
    i += 1

In [6]:
train_encoded_data = train_encoded_data[["source_system_tab", "source_screen_name", "source_type",
                                         "song_length", "genre_ids", "language", "city", "age_group",
                                         "gender", "registered_via", "target"]]
train_encoded_data.head(2)

,source_system_tab,source_screen_name,source_type,song_length,genre_ids,language,city,age_group,gender,registered_via,target
0,2,7,7,206471.0,21,52.0,1,0,0,7,1
1,4,8,5,284584.0,105,52.0,13,1,1,9,1


In [7]:
test_encoded_data = test_encoded_data[["source_system_tab", "source_screen_name", "source_type",
                                         "song_length", "genre_ids", "language", "city", "age_group",
                                         "gender", "registered_via"]]
test_encoded_data.head(2)

,source_system_tab,source_screen_name,source_type,song_length,genre_ids,language,city,age_group,gender,registered_via
0,4,8,4,224130.0,33,3.0,1,0,0,7
1,4,8,4,320470.0,34,3.0,1,0,0,7


In [8]:
# Create features and targets datasets
features = train_encoded_data.drop("target", axis = 1)
targets = pd.DataFrame(train_encoded_data['target'])

In [9]:
features.head(2)

,source_system_tab,source_screen_name,source_type,song_length,genre_ids,language,city,age_group,gender,registered_via
0,2,7,7,206471.0,21,52.0,1,0,0,7
1,4,8,5,284584.0,105,52.0,13,1,1,9


In [10]:
targets.head(2)

,target
0,1
1,1


# Split Data in Trainning and Testing datasets

In [11]:
# Split into 70% training and 30% testing set
X, X_test, y, y_test = train_test_split(features, targets, test_size = 0.3, random_state = 42)

print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)

(5164192, 10)
(2213226, 10)
(5164192, 1)
(2213226, 1)


# Scale data

In [12]:
# Create the scaler object with a range of 0-1
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

# Fit on the training data
scaler.fit(X)

# Transform both the training and testing data
X = scaler.transform(X)
X_test = scaler.transform(X_test)

/anaconda3/envs/Finance/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [13]:
# Convert labels to one-hot-encoding
y = to_categorical(y)
y_test = to_categorical(y_test)

# Apply Deep Learning Model

In [46]:
# Create model and add layers
model = Sequential()
model.add(Dense(units= 100, activation='relu', input_dim=10))
model.add(Dense(units= 100, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))

In [47]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
model.fit(
    X,
    y,
    epochs=15,
    batch_size = 1000,
    shuffle=True,
    verbose=2
)

Epoch 1/15
 - 21s - loss: 0.6573 - acc: 0.6219
Epoch 2/15
 - 27s - loss: 0.6521 - acc: 0.6260
Epoch 3/15
 - 21s - loss: 0.6510 - acc: 0.6268
Epoch 4/15
 - 20s - loss: 0.6503 - acc: 0.6275
Epoch 5/15
 - 20s - loss: 0.6497 - acc: 0.6279
Epoch 6/15
 - 21s - loss: 0.6493 - acc: 0.6283
Epoch 7/15
 - 21s - loss: 0.6489 - acc: 0.6286
Epoch 8/15
 - 21s - loss: 0.6485 - acc: 0.6288
Epoch 9/15
 - 20s - loss: 0.6483 - acc: 0.6291
Epoch 10/15
 - 21s - loss: 0.6481 - acc: 0.6292
Epoch 11/15
 - 21s - loss: 0.6478 - acc: 0.6294
Epoch 12/15
 - 22s - loss: 0.6476 - acc: 0.6295
Epoch 13/15
 - 21s - loss: 0.6475 - acc: 0.6297
Epoch 14/15
 - 21s - loss: 0.6473 - acc: 0.6297
Epoch 15/15
 - 21s - loss: 0.6472 - acc: 0.6299


## Quantify our Trained Model

In [49]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

 - 45s - loss: 0.6472 - acc: 0.6296
Normal Neural Network - Loss: 0.647221459851231, Accuracy: 0.6296008825302124


## Make Predictions with Kaggle Test Dataset

## Scale Data

In [27]:
# Transform both the training and testing data
test_encoded_data_scaled = scaler.transform(test_encoded_data)

## Predict Targets

In [28]:
predictions = model.predict_classes(test_encoded_data_scaled)

In [30]:
predictions[:5]

array([1, 1, 0, 0, 0])

In [33]:
predicted_data = pd.DataFrame(test_data['id'])

In [34]:
predicted_data["target"] = predictions

In [35]:
predicted_data.head()

,id,target
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0


In [36]:
predicted_data.to_csv("sample_submission.csv",index=False)